In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [131]:
import json

from operator import add

In [3]:
reviews = sc.textFile('/data/yelp/review_sample').repartition(10)
users = sc.textFile('/data/yelp/user_sample').repartition(10)

In [158]:
def fetch_user_id_useful(review_line):
    dct_review = json.loads(review_line)
    return dct_review['user_id'], dct_review['useful']


def fetch_user_id_friends(user_line):
    dct_user = json.loads(user_line)
    return [(friend, dct_user['user_id']) for friend in dct_user['friends'].strip().split(', ')]


def fetch_user_id_friends_number(user_line):
    dct_user = json.loads(user_line)
    friends_list = dct_user['friends'].strip().split(', ')
    if friends_list[0] == 'None':
        friends_number = 0
    else:
        friends_number = len(friends_list)
    return dct_user['user_id'], friends_number


def sorting_criteria(merge_line):
    usefuls, influence = merge_line[1]
    influence = 0 if influence is None else influence
    return usefuls * influence

In [28]:
reviews_rdd0 = reviews.map(lambda x: fetch_user_id_useful(x))
reviews_rdd1 = reviews_rdd0.groupByKey().mapValues(list)
reviews_rdd2 = reviews_rdd1.map(lambda x: (x[0], sum(sorted(x[1], reverse=True)[:5])))

In [119]:
users_rdd0 = users.flatMap(lambda x: fetch_user_id_friends(x))
users_rdd1 = users.map(lambda x: fetch_user_id_friends_number(x))
users_rdd2 = users_rdd1.leftOuterJoin(users_rdd0).map(lambda x: (x[1][1], x[1][0])).reduceByKey(add)

In [162]:
merge_rdd = reviews_rdd2.leftOuterJoin(users_rdd2).sortBy(lambda x: sorting_criteria(x), ascending=False)

In [167]:
top10 = merge_rdd.take(10)
for influencer in top10:
    _id, (usefuls, influence) = influencer
    print(f'{_id}\t{usefuls}\t{influence}')

Hi10sGSZNxQH3NLyWSZ1oA	327	10448
3QnoTcrxuafMCoTzW_AH7A	253	10541
95GOfKoCDfeGkX2l8n8Lfw	882	2850
sraTrYU-7q_bQ0TxQrLJvQ	249	9151
uIjj7EIVBU4kGNgmKPO02A	160	13115
VtiE3eXNYtH-tb0aFCF1Pw	267	7220
X_DkwPTzdO_VWzUcbUXREg	297	5595
6tbXpUIU6upoeqWNDo9k_A	146	10836
QnIrewrXxFnssw_gG1gF-Q	149	9984
InInjzaZxzQ8_suxZEAP5g	306	4358
